Partitioning data into 'Train'(65%), 'Val'(15%) and 'Test'(20%)

Approach - 

1. When we apply group by clause over the 'reading_age' column, we get 80 different age-ranges in which the data can be segregated. 
2. First gather the indices according to age-range. 
3. If count of indices is >= 3, part the data with the ratio of 13:3:4 (train, validation, test)
4. if count is ==2,  part the data into Train and test equally.  
5. if count  == 1, put the data into train. 

In [1]:
from google.colab import drive 
drive.mount('/gdrive')


Mounted at /gdrive


In [2]:
import pandas as pd
book_synopsis = pd.read_csv('/gdrive/My Drive/MSC_Thesis_Project/Data/bookSynopsis_ageRange_clean.csv')

df1 = book_synopsis.groupby(['Reading_age'])

print(df1.count())

             Unnamed: 0  Title  Desc  lower_age  upper_age
Reading_age                                               
0-5                 691    691   691        691        691
11-18               212    212   212        212        212
6-10                678    678   678        678        678


In [7]:
book_synopsis

,Unnamed: 0,Title,Desc,Reading_age,lower_age,upper_age
0,0,A TRICK TRIKE,Little Ross was sad. He didnt have a nice new ...,0-5,0,5
1,1,ABE THE SERVICE DOG,Abe was a real Service Dog who dedicated his l...,0-5,0,5
2,2,BUBBLE FUN,"When the last bubble popped, he was gone but a...",0-5,0,5
3,3,CAPTAIN FANTASTIC,"Dr. Zobs evil plans had failed once again, th...",0-5,0,5
4,4,DO YOU WONDER WHY BUGS ARE GOOD?,Those little beauties have inspired so many st...,0-5,0,5
...,...,...,...,...,...,...
1576,1577,You're God's Girl!: A Devotional for Tweens,\nDiscover the True You! Who is the real you? ...,6-10,6,10
1577,1578,You're My Little Cuddle Bug,\nCelebrate your little cuddle bug with this c...,0-5,0,5
1578,1579,Your Baby's First Word Will Be DADA,"\nYour baby's first word will be . . .""Dada!""R...",0-5,0,5
1579,1580,Your Diary - Sparkly Lock & Keys - Girls 8+ Jo...,"\nBy You, about You, for You! Your Diary is al...",6-10,6,10


In [ ]:
##
##Below function is written to split the available data into three partitions of train, validate and test.  
##
def split_three(lst, ratio):
    import numpy as np

    train_r, val_r, test_r = ratio
    assert(np.sum(ratio) == 1.0)  # makes sure the splits make sense
    # note we only need to give the first 2 indices to split, the last one it returns the rest of the list or empty
    indicies_for_splitting = [int(len(lst) * train_r), int(len(lst) * (train_r+val_r))]
    train, val, test = np.split(lst, indicies_for_splitting)
    return train, val, test

In [3]:
#import pandas as pd 
import random

## extracting the row indices from book_synopsis_ageRange_clean.csv by grouping 'Reading_age' column.
## storing the values in age_range_indices dataframe 

book_synopsis = pd.read_csv('/gdrive/My Drive/MSC_Thesis_Project/Data/bookSynopsis_ageRange_clean.csv')

df = book_synopsis.groupby(['Reading_age'])

col_names = ['Reading_age','Row_indices']
age_range_indices = pd.DataFrame(columns= col_names)

index = []
indices= []
count=0
for key, item in df:
    indices=[]                        #initializing indices to zero
    index= df.get_group(key).index    #get key from group and retrieve the indices 
    
    for i in index:
        indices.append(i) 
    
    age_range_indices = age_range_indices.append({
                                                  'Reading_age': key,
                                                  'Row_indices': indices
                                              }, ignore_index= True)
   
age_range_indices     #['Row_indices']


,Reading_age,Row_indices
0,0-5,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,11-18,"[26, 32, 37, 38, 39, 40, 41, 48, 49, 50, 66, 7..."
2,6-10,"[23, 24, 27, 29, 30, 31, 33, 34, 36, 45, 47, 5..."


In [ ]:

data_partition= {}
data_partition['train']=[]
data_partition['val']= []
data_partition['test']=[]

##percentage of partitioning for the available data 
train_pc = 0.65           
val_pc = 0.15
test_pc = 0.2

ids=[]

for i,ids in age_range_indices['Row_indices'].items():     
    if len(ids) == 1: 
       data_partition['train'].append(ids[0]) 
    elif len(ids) == 2: 
       data_partition['train'].append(ids[0])
       data_partition['test'].append(ids[1])    
    else:        
       #permute list of ids -
       random.shuffle(ids)                        
       train_ids, val_ids, test_ids = split_three(ids,[train_pc, val_pc, test_pc])            
       data_partition['train'].extend(train_ids)
       data_partition['test'].extend(test_ids)
       data_partition['val'].extend(val_ids)     
    


In [ ]:
len(data_partition['val'])

237

In [ ]:
# Save the train, validation and test splits
!pip install numpyencoder
from numpyencoder import NumpyEncoder
import json

with open('/gdrive/My Drive/MSC_Thesis_Project/Data/train_val_test_indices.json', 'w') as fp:
    json.dump(data_partition, fp, cls=NumpyEncoder)


In [18]:
book_synopsis

,Unnamed: 0,Title,Desc,Reading_age,lower_age,upper_age
0,0,A TRICK TRIKE,Little Ross was sad. He didnt have a nice new ...,0-5,0,5
1,1,ABE THE SERVICE DOG,Abe was a real Service Dog who dedicated his l...,0-5,0,5
2,2,BUBBLE FUN,"When the last bubble popped, he was gone but a...",0-5,0,5
3,3,CAPTAIN FANTASTIC,"Dr. Zobs evil plans had failed once again, th...",0-5,0,5
4,4,DO YOU WONDER WHY BUGS ARE GOOD?,Those little beauties have inspired so many st...,0-5,0,5
...,...,...,...,...,...,...
1576,1577,You're God's Girl!: A Devotional for Tweens,\nDiscover the True You! Who is the real you? ...,6-10,6,10
1577,1578,You're My Little Cuddle Bug,\nCelebrate your little cuddle bug with this c...,0-5,0,5
1578,1579,Your Baby's First Word Will Be DADA,"\nYour baby's first word will be . . .""Dada!""R...",0-5,0,5
1579,1580,Your Diary - Sparkly Lock & Keys - Girls 8+ Jo...,"\nBy You, about You, for You! Your Diary is al...",6-10,6,10


In [20]:
import numpy as np
# Using the code in cell-6 of extract_statistical_features_bert.ipynb, 
# write code which maps 0-5 to a number 0, 6-11 to 1, 12-18 to 2
# then writes a 1581 x 1 array to a file called targets_file.npy

df = book_synopsis
lower = 0
upper = 0
x = []

for i,ids in df['lower_age'].items():
  lower = df['lower_age'][i]
  upper = df['upper_age'][i]  
  if ( lower == 0 and upper == 5):
    x.append(0)
  elif ( lower == 6 and upper == 10):
    x.append(1)
  elif ( lower == 11 and upper == 18):  
    x.append(2)
  
  
X= np.array(x)

# Saving the target labels 
np.save('/gdrive/My Drive/MSC_Thesis_Project/Data/target_labels', X)  #will be saved as .npy file
